In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


%matplotlib inline


In [2]:
# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

In [3]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [4]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


### Getting to know the data

In [5]:
reviews['rating'].shape

(863866,)

In [6]:
#The number of movies in the dataset
print('There are {} number of movies in the dataset'.format(movies['movie'].shape[0]))
print()
print('There are {} number of ratings in the dataset'.format(reviews['rating'].shape[0]))

There are 35479 number of movies in the dataset

There are 863866 number of ratings in the dataset


In [7]:
movies['genre'].value_counts()

Drama                                        3602
Comedy                                       2091
Documentary                                  1443
Comedy|Drama                                 1371
Drama|Romance                                1199
                                             ... 
Action|Fantasy|Horror|Romance|Thriller          1
Adventure|Comedy|Music|Sci-Fi                   1
Documentary|Biography|History|News|War          1
Documentary|Biography|Crime|History|Sport       1
Action|Adventure|Crime|Sci-Fi|Thriller          1
Name: genre, Length: 2736, dtype: int64

In [8]:
# a function to split elements in genre
genres = []
for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass

print("The list of different movie genres are: {}".format(set(genres)))
print()
print("The number of different genres in the data set is {}".format(len(set(genres))))

The list of different movie genres are: {'Sport', 'Mystery', 'Drama', 'Sci-Fi', 'Comedy', 'Reality-TV', 'Documentary', 'Animation', 'Game-Show', 'Short', 'Family', 'War', 'Horror', 'Western', 'Thriller', 'Music', 'Film-Noir', 'Action', 'Adult', 'Fantasy', 'Crime', 'Talk-Show', 'Romance', 'News', 'Biography', 'Adventure', 'History', 'Musical'}

The number of different genres in the data set is 28


In [9]:
print("The number of unique users in the dataset is {}".format(reviews['user_id'].nunique()))
print()
print("The number of missing ratings in the reviews dataset is {}".format(reviews['rating'].isnull().sum()))

The number of unique users in the dataset is 67353

The number of missing ratings in the reviews dataset is 0


In [10]:
print("The average rating given across all ratings is {}".format(reviews['rating'].mean()))
print()
print("The minimum rating given across all ratings is {}".format(reviews['rating'].min()))
print()
print("The maximum rating given across all ratings is {}".format(reviews['rating'].max()))

The average rating given across all ratings is 7.315877693994207

The minimum rating given across all ratings is 0

The maximum rating given across all ratings is 10


### Data cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy the genre column with 1's and 0's

#### Reviews
* Create a date out of time stamp


1) **Pull the date from the movie title and create new column**

In [11]:
movies['movie'][0]

'Edison Kinetoscopic Record of a Sneeze (1894)'

In [12]:
movies['movie'][0][-5:-1]

'1894'

In [13]:
# pull date if it exists and create a new column 'date'
create_date = lambda val: val[-5:-1] if val[-1] == ')' else np.nan

# apply the function to pull the date
movies['date'] = movies['movie'].apply(create_date)

movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


2) **Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)**

In [14]:
# Return century of movie as a dummy column
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0
        
# Apply function
for yr in ['18', '19', '20']:
    movies[str(yr) + "00's"] = movies['date'].apply(add_movie_year)

In [15]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


3) **Dummy the genre column with 1's and 0's**

In [16]:
def split_genres(val):
    try:
        if val.find(g) > -1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0
    
for g in genres:
    movies[g] = movies['genre'].apply(split_genres)

In [17]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Documentary,Short,Horror,...,Thriller,Animation,Music,Musical,Film-Noir,Adult,Talk-Show,News,Reality-TV,Game-Show
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


4) **Create a date out of time stamp**

In [20]:
#change timstamp to datestamp
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [21]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-05 15:00:50
1,2,0208092,5,1586466072,2020-04-09 15:01:12
2,2,0358273,9,1579057827,2020-01-14 21:10:27
3,2,10039344,5,1578603053,2020-01-09 14:50:53
4,2,6751668,9,1578955697,2020-01-13 16:48:17


In [26]:
reviews['month'] = pd.DatetimeIndex(reviews['date']).month
reviews['year'] = pd.DatetimeIndex(reviews['date']).year
reviews.head()

,user_id,movie_id,rating,timestamp,date,month,year
0,1,0114508,8,1381006850,2013-10-05 15:00:50,10,2013
1,2,0208092,5,1586466072,2020-04-09 15:01:12,4,2020
2,2,0358273,9,1579057827,2020-01-14 21:10:27,1,2020
3,2,10039344,5,1578603053,2020-01-09 14:50:53,1,2020
4,2,6751668,9,1578955697,2020-01-13 16:48:17,1,2020


In [28]:
reviews = pd.get_dummies(reviews, columns = ['month','year'])
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020
0,1,0114508,8,1381006850,2013-10-05 15:00:50,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2,0208092,5,1586466072,2020-04-09 15:01:12,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,2,0358273,9,1579057827,2020-01-14 21:10:27,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,10039344,5,1578603053,2020-01-09 14:50:53,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2,6751668,9,1578955697,2020-01-13 16:48:17,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
movies.to_csv('df_movies_clean',index=False)
reviews.to_csv('df_reviews_clean',index=False)